In [1]:
from math import sqrt
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,RidgeCV,Lasso,LassoCV,ElasticNet,ElasticNetCV
from sklearn import metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error
from sklearn.model_selection import KFold

import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib  inline

In [10]:
train =  pd.read_csv('trainofbike.csv',  parse_dates = ['datetime'])
test = pd.read_csv('testofbike.csv', parse_dates = ['datetime'])

In [3]:
#datetimeの変換
import datetime as dt
train['day'] = train.datetime.dt.day
train['hour'] = train.datetime.dt.hour
train['weekday'] = train.datetime.dt.weekday
train['year'] = train.datetime.dt.year
train['month'] = train.datetime.dt.month
test['day'] = test.datetime.dt.day
test['hour'] = test.datetime.dt.hour
test['weekday'] = test.datetime.dt.weekday
test['year'] = test.datetime.dt.year
test['month'] = test.datetime.dt.month

In [4]:
train_working_0 = train[train['workingday'] == 0]
train_working_1 = train[train['workingday'] == 1]
train_0 = train_working_0.drop(columns = ['datetime', 'casual', 'registered', 'count'])
train_1 = train_working_1.drop(columns = ['datetime', 'casual', 'registered', 'count'])
train_y = train['count']
train_y_0 = train_working_0['count']
train_y_1 = train_working_1['count']
train_y_log = np.log1p(train_y)
train_y_0_log = np.log1p(train_y_0)
train_y_1_log = np.log1p(train_y_1)

In [4]:
train_0 = train.drop(columns = ['datetime', 'casual', 'registered', 'count'])
train_y = train['count']
train_y_log = np.log1p(train_y)

t_w_0 = train[train['workingday'] == 0]
t_w_0_0 = t_w_0.drop(columns = ['datetime', 'casual', 'registered', 'count'])
t_w_0_y = t_w_0['count']
t_w_0_y_log = np.log1p(t_w_0_y)

t_w_1 = train[train['workingday'] == 1]
t_w_1_0 = t_w_1.drop(columns = ['datetime', 'casual', 'registered', 'count'])
t_w_1_y = t_w_1['count']
t_w_1_y_log = np.log1p(t_w_1_y)

In [8]:
# ENet 可視化(count log用)
alphas = 10**np.linspace(-5, 10, 500)

Elastic_cv = ElasticNetCV(alphas=alphas, cv=4)
Elastic_cv.fit(t_w_0_0, t_w_0_y_log)
print(Elastic_cv.alpha_)

kf = KFold(n_splits=4, shuffle=True, random_state=1).split(t_w_0_0, t_w_0_y_log)
kf_ = list(kf)

cnt = 0
sum = 0
for trainK, testK in kf_:
    cnt += 1
    from sklearn.linear_model import ElasticNet,ElasticNetCV
    Elastic = ElasticNet(alpha=Elastic_cv.alpha_)
    Elastic.fit(train_0.loc[trainK], train_y_log.loc[trainK])
    Elastic_train_y_pred_log = Elastic.predict(train_0.loc[testK])
    y_pred = np.exp(Elastic_train_y_pred_log)-1
    y_pred[y_pred < 0] = 0
    y_true = train_y.loc[testK]
    print(f"Fold:{cnt}")
    print('RMSLE:', np.sqrt(mean_squared_log_error(y_true, y_pred)))
    score = np.sqrt(mean_squared_log_error(y_true, y_pred))
    sum += score
    diff = y_true - y_pred
    print(diff.sum())

mean_score = sum / 4
print(f"mean_score_RMSLE = {mean_score}")

C:\Users\sibuya yuuki\miniconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.0600518671794816, tolerance: 0.6123271051724647
  model = cd_fast.enet_coordinate_descent(


1e-05
Fold:1
RMSLE: 0.9721919568441822
17505.95740517213
Fold:2
RMSLE: 1.0041243258631338
32182.266572409542
Fold:3
RMSLE: 0.9729998497891655
19929.508564773987
Fold:4
RMSLE: 0.999785660037192
23899.582877600445
mean_score_RMSLE = 0.9872754481334184


In [11]:
train_0[train_0.index == 3433]

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,day,hour,weekday,year,month


In [9]:
train

array([   0,    1,    2, ..., 3469, 3472, 3473])

In [9]:
# ENet 可視化(count log用)
alphas = 10**np.linspace(-5, 10, 500)

Elastic_cv = ElasticNetCV(alphas=alphas, cv=4)
Elastic_cv.fit(t_w_1_0, t_w_1_y_log)
print(Elastic_cv.alpha_)

kf = KFold(n_splits=4, shuffle=True, random_state=1).split(t_w_1_0, t_w_1_y_log)
kf_ = list(kf)

cnt = 0
sum = 0
for trainK, testK in kf_:
    cnt += 1
    from sklearn.linear_model import ElasticNet,ElasticNetCV
    Elastic = ElasticNet(alpha=Elastic_cv.alpha_)
    Elastic.fit(train_0.loc[trainK], train_y_log.loc[trainK])
    Elastic_train_y_pred_log = Elastic.predict(train_0.loc[testK])
    y_pred = np.exp(Elastic_train_y_pred_log)-1
    y_pred[y_pred < 0] = 0
    y_true = train_y.loc[testK]
    print(f"Fold:{cnt}")
    print('RMSLE:', np.sqrt(mean_squared_log_error(y_true, y_pred)))
    score = np.sqrt(mean_squared_log_error(y_true, y_pred))
    sum += score
    diff = y_true - y_pred
    print(diff.sum())

mean_score = sum / 4
print(f"mean_score_RMSLE = {mean_score}")

0.0035901303770070683
Fold:1
RMSLE: 1.045663532683892
64694.449289051816
Fold:2
RMSLE: 1.0045994860564593
55774.73375240294
Fold:3
RMSLE: 1.018944669148241
61180.48791337009
Fold:4
RMSLE: 1.0297734938142242
68221.9938754146
mean_score_RMSLE = 1.0247452954257041


In [ ]:
test_working_0 = test[test['workingday'] == 0]
test_working_1 = test[test['workingday'] == 1]
test_0 = test_working_0.drop(columns = ['datetime'])
test_1 = test_working_1.drop(columns = ['datetime'])
test_datetime_0 = test_working_0['datetime']
test_datetime_1 = test_working_1['datetime']